In [1]:
pip install pandas translate pylatexenc dl_translate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.4 MB/s eta 0:00:00
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136818 sha256=5e8def8097593551bda1affed397b1b2fb6a52e81c99fb1cc898c2d71c53a263
  Stored in directory: /root/.cache/pip/wheels/d3/31/8b/e09b0386afd80cfc556c00408c9aeea5c35c4d484a9c762fd5
Successfully built pylatexenc


In [2]:
# попытка что-то поделать
import pandas as pd
from translate import Translator

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# занимает много времени, лучше делать один раз
# заранее берём только те столбцы, в которых содержится текст (остальные не рассматриваем)
df = pd.read_json('/content/drive/MyDrive/questions_only.json')[['body_markdown','tags']]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 121700 entries, 0 to 121699
Data columns (total 2 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   body_markdown  121700 non-null  object
 1   tags           121700 non-null  object
dtypes: object(2)
memory usage: 2.8+ MB


In [5]:
import dl_translate as dlt
# **ОЧЕНЬ** долго загружается, если впервые -- дать подумать
# device="auto" для выбора GPU при возможности, у меня с этим проблемы, т.ч. в целом можно оставить дефолт
# дефолтная модель -- не "mbart50", доки: github.com/xhluca/dl-translate
mt = dlt.TranslationModel("mbart50")

In [6]:
pip install tensorflow

In [7]:
import tensorflow as tf
print(tf.__version__)

2.14.0


In [8]:
import tensorflow as tf

# Проверяем доступность GPU
if tf.test.gpu_device_name():
    print('GPU доступен: {}'.format(tf.test.gpu_device_name()))
else:
    print('GPU не доступен. Проверьте настройки среды выполнения.')

GPU доступен: /device:GPU:0


In [9]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Измените "0" на номер доступного GPU
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"



In [10]:
pip install Transformers nltk

In [11]:
import nltk
nltk.download("punkt")



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [12]:
import torch
torch.cuda.empty_cache()

In [13]:
rows = []
for i, b_ms in enumerate(df['body_markdown']):
    if i == 1000:
      break
    row = {}
    row['markdown'] = b_ms
    torch.cuda.empty_cache()
    sentences = nltk.sent_tokenize(b_ms, "english")
    translated_sentences = []
    batch_size = 4  # Уменьшите batch size
    for j in range(0, len(sentences), batch_size):
        batch = sentences[j:j+batch_size]
        translated_batch = mt.translate(batch, source=dlt.lang.ENGLISH, target=dlt.lang.RUSSIAN)
        translated_sentences.extend(translated_batch)
    row['markdown_rus'] = " ".join(translated_sentences)
    torch.cuda.empty_cache()
    rows.append(row)
    print(i+1,".", ((i+1) / 1217) )


1 . 0.0008216926869350862
2 . 0.0016433853738701725
3 . 0.0024650780608052587
4 . 0.003286770747740345
5 . 0.004108463434675432
6 . 0.0049301561216105174
7 . 0.005751848808545604
8 . 0.00657354149548069
9 . 0.007395234182415777
10 . 0.008216926869350863
11 . 0.009038619556285949
12 . 0.009860312243221035
13 . 0.010682004930156122
14 . 0.011503697617091208
15 . 0.012325390304026294
16 . 0.01314708299096138
17 . 0.013968775677896467
18 . 0.014790468364831553
19 . 0.015612161051766639
20 . 0.016433853738701727
21 . 0.01725554642563681
22 . 0.018077239112571898
23 . 0.018898931799506986
24 . 0.01972062448644207
25 . 0.020542317173377157
26 . 0.021364009860312245
27 . 0.02218570254724733
28 . 0.023007395234182416
29 . 0.0238290879211175
30 . 0.024650780608052588
31 . 0.025472473294987676
32 . 0.02629416598192276
33 . 0.027115858668857847
34 . 0.027937551355792935
35 . 0.02875924404272802
36 . 0.029580936729663106
37 . 0.030402629416598194
38 . 0.031224322103533278
39 . 0.03204601479046836
4

RuntimeError: ignored

In [28]:
import csv

# Имя файла CSV
csv_file = 'output.csv'

# Открытие файла CSV в режиме записи
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    # Создание объекта записи CSV
    writer = csv.DictWriter(file, fieldnames=rows[0].keys(), escapechar='\\')

    # Запись заголовков
    writer.writeheader()

    # Запись данных из листа словарей
    writer.writerows(rows)

In [32]:
df = pd.read_csv('/content/output.csv')
df.head(10)

,markdown,markdown_rus
0,$C$ is a convex set and $ x \\in R^d. P_c(x) $...,$C$ — выпуклый набор и $ x \\in R^d. P_c(x) $ ...
1,Let $\\frak{g}$ be a simple complex Lie algebr...,Пусть $\\frak{g}$ является простым комплексом ...
2,Any ideas how to evaluate this integral over t...,"Есть какие-то идеи о том, как оценить этот инт..."
3,I currently have an issue understanding a line...,В настоящее время у меня есть проблема в поним...
4,Sorry if this is a very basic question. I&#39;...,"Извините, если это очень простой вопрос. Я тол..."
5,Every time that one hears about Game Theory im...,"Каждый раз, когда кто-то слышит о теории игры ..."
6,I am studying for an exam for work which is co...,"Я изучаю для экзамена на работу, которая охват..."
7,$$\\lim_{n \\to \\infty}\\ln\\Bigg(\\prod_{j=1...,$$\\lim_{n \\to \\infty}\\ln\\Bigg(\\prod_{j=1...
8,Consider the p-adic field $ \\ \\mathbb{Q}_p \...,Рассмотрим поле p-адического $ \\ \\mathbb{Q}_...
9,I have an integral that looks like an incomple...,"У меня интеграл, который выглядит как неполная..."
